# This note book is Modified for Haruki's learning. 
This notebook uses the model created in pretrain any model notebook.

1. Pretrain Roberta Model: https://www.kaggle.com/maunish/clrp-pytorch-roberta-pretrain
2. Finetune Roberta Model: this notebook, <br/>
   Finetune Roberta Model TPU: https://www.kaggle.com/maunish/clrp-pytorch-roberta-finetune-tpu
3. Inference Notebook: https://www.kaggle.com/maunish/clrp-pytorch-roberta-inference
4. Roberta + SVM: https://www.kaggle.com/maunish/clrp-roberta-svm

In [ ]:
!pip install accelerate

In [ ]:
import os
#garbage collectionメモリの開放
import gc

import sys
import math
import time

#pythonでシークバーを表示する
import tqdm

import random
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_squared_error

#Stratified KFold -> 層化抽出K分割公差検証
#目的変数の偏りが保持されるようにK分割を実施する
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from accelerate import Accelerator
from transformers import (AutoModel,AutoConfig,
                          AutoTokenizer,get_cosine_schedule_with_warmup)

#色関係?EDA用?
from colorama import Fore, Back, Style
r_ = Fore.RED
b_ = Fore.BLUE
c_ = Fore.CYAN
g_ = Fore.GREEN
y_ = Fore.YELLOW
m_ = Fore.MAGENTA
sr_ = Style.RESET_ALL

In [ ]:
#ここから再開
train_data = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test_data = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
sample = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')

#print(train_data.head())
#exerptの改行情報を削除
train_data['excerpt'] = train_data['excerpt'].apply(lambda x: x.replace('\n',''))

#np.floorは少数点以下を四捨五入する関数. train dataのレコード数の対数+1を整数化
num_bins = int(np.floor(1 + np.log2(len(train_data))))

#binsで指定した数の群にtargetによって分割する
#labels=Falseでは出力の値域は非表示となり, 0始まりの群のidのみがoutputされる
#loc methodは[行,列]要素の順番で指定してdataframeの要素を抽出する.
#以下の例だと、すべての行のbins列に対して、群番号を(新規列)binsに格納している
train_data.loc[:,'bins'] = pd.cut(train_data['target'],bins=num_bins,labels=False)

#bins/targetを夫々numpyクラスに変換
bins = train_data.bins.to_numpy()
target = train_data.target.to_numpy()

#print("bins", type(bins))
#print("target", target)

#二乗和平均誤差関数を定義
def rmse_score(y_true,y_pred):
    return np.sqrt(mean_squared_error(y_true,y_pred))

In [ ]:
print(train_data.head())

In [ ]:
#HP設定, HPの各詳細もあとで追いかける
config = {
    'lr': 2e-5,
    'wd':0.01,
    'batch_size':8,
    'valid_step':10,
    'max_len':256,
    'epochs':3,
    'nfolds':5,
    'seed':42,
    'model_path':'../input/clrp-pytorch-roberta-pretrain-roberta-large/clrp_roberta_large',
}

#nfolds分の格納用dirを作成
for i in range(config['nfolds']):
    os.makedirs(f'model{i}',exist_ok=True)

#seed設定の関数
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

#seedを42に設定
seed_everything(seed=config['seed'])

#FOLD列を追加して仮に-1を一律格納
train_data['Fold'] = -1
#層化抽出khold, シャッフル有効, seed固定, 
#kfoldクラスを使ったid抽出は以下が分かりやすい.
#https://blog.amedama.jp/entry/2018/06/21/235951
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold.split
#Stratified Kfoldにおけるsplitにおいてyは層化抽出に必須のパラメタ, binsを層化抽出する
kfold = StratifiedKFold(n_splits=config['nfolds'],shuffle=True,random_state=config['seed'])
for k , (train_idx,valid_idx) in enumerate(kfold.split(X=train_data,y=bins)):
    train_data.loc[valid_idx,'Fold'] = k

In [ ]:
train_data.head()

In [ ]:
plt.figure(dpi=100)
#seabornはこの形だけでプロットできるのすごい...
#binsで区切った適正年齢群は、ほぼシグマプロットの様な形状をしている.
sns.countplot(train_data.bins);

In [ ]:
#Dataset Class定義、初期値としてexcerptとtargets、max_len, tokenizerを持つ。
#Class定義におけるダブルアンダーバーに特別な意味があったか.

class CLRPDataset(Dataset):
    def __init__(self,df,tokenizer,max_len=128):
        self.excerpt = df['excerpt'].to_numpy()
        self.targets = df['target'].to_numpy()
        self.max_len = max_len
        self.tokenizer = tokenizer

    #get itemでは、tokenizerに割り当てられる番号とtarget値が出てくる？
    #getitemについてhttps://qiita.com/gyu-don/items/bde192b129a7b1b8c532
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
                                return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        
        target = torch.tensor(self.targets[idx],dtype=torch.float) 
        return encode, target
    
    #excerptの長さ
    def __len__(self):
        return len(self.excerpt)

In [ ]:
#AttentionHeadクラスの定義
class AttentionHead(nn.Module):
    def __init__(self, in_features, hidden_dim):
        super().__init__()
        self.in_features = in_features
        self.middle_features = hidden_dim
        self.W = nn.Linear(in_features, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
        self.out_features = hidden_dim
    #forwardではattentionを計算して、context_vectorを返す
    def forward(self, features):
        #W(argument)でargumentに対してnn.linearを実施, hidden_dimに変換して、tanh変換
        att = torch.tanh(self.W(features))
        #入力=>隠れ層へ全結合=>tanh変換をscoreに線形変換
        score = self.V(att)
        #attention重みをsoftmax関数で作成
        attention_weights = torch.softmax(score, dim=1)
        #context_vectorはvalueにattentionをつけたもの
        context_vector = attention_weights * features
        context_vector = torch.sum(context_vector, dim=1)

        return context_vector

In [ ]:
class Model(nn.Module):
    #Model本体は一つ前のnotebookで作成したmodelをpathで参照する形式
    def __init__(self,path):
        super(Model,self).__init__()
        #robertaモデルを読み込み
        self.roberta = AutoModel.from_pretrained(path)
        #configを読み込み？
        self.config = AutoConfig.from_pretrained(path)
        #attentionHeadを読み込み
        self.head = AttentionHead(self.config.hidden_size,self.config.hidden_size)
        #Dropoutを0.1に設定
        self.dropout = nn.Dropout(0.1)
        #1次元への全結合層?
        self.linear = nn.Linear(self.config.hidden_size,1)

    def forward(self,**xb):
        #xbをモデルに入れる**が引数にあるのは任意の数の引数を指定できる意味
        #input xbをrobertaモデルに解釈させて、headを取って、dropoutさせて、全結合してxを返す
        #forwardはattention自体を返す=予測値？
        x = self.roberta(**xb)[0]
        x = self.head(x)
        x = self.dropout(x)
        x = self.linear(x)
        return x

In [ ]:
#fold数を指定して、runする？
def run(fold,verbose=True):
    #loss関数の設定
    def loss_fn(outputs,targets):
        outputs = outputs.view(-1)
        targets = targets.view(-1)
        return torch.sqrt(nn.MSELoss()(outputs,targets))
    
    #そのまま？
    def train_and_evaluate_loop(train_loader,valid_loader,model,loss_fn,optimizer,epoch,fold,best_loss,valid_step=10,lr_scheduler=None):
        train_loss = 0
        for i, (inputs1,targets1) in enumerate(train_loader):
            #modelをtrainする宣言
            model.train()
            #gradienｔの初期化
            optimizer.zero_grad()
            #inputs1のkey valを変形してinputs1に再代入
            inputs1 = {key:val.reshape(val.shape[0],-1) for key,val in inputs1.items()}
            #outputはmodelで処理したinputs1
            outputs1 = model(**inputs1)
            #loss1は誤差関数
            loss1 = loss_fn(outputs1,targets1)
            #損失関数に関して、伝搬への影響を微分計算
            loss1.backward()
            #学習率と最適化手法に基づいて学習を実施する
            optimizer.step()
            
            #loss1を代入 書く train_loader要素に対してforループを回しているので.
            train_loss += loss1.item()
            
            #学習率を動的に変化させるscheduler.
            if lr_scheduler:
                lr_scheduler.step()
            
            #evaluating for every valid_step
            #特定のタイミングで評価, valid_stepまたはループのケツ
            if (i % valid_step == 0) or ((i + 1) == len(train_loader)):
                model.eval()
                valid_loss = 0
                with torch.no_grad():
                    for j, (inputs2,targets2) in enumerate(valid_loader):
                        inputs2 = {key:val.reshape(val.shape[0],-1) for key,val in inputs2.items()}
                        outputs2 = model(**inputs2)
                        loss2 = loss_fn(outputs2,targets2)
                        valid_loss += loss2.item()
                     
                    valid_loss /= len(valid_loader)
                    if valid_loss <= best_loss:
                        #評価タイミングで比較して, inputされたbest_lossに対して,
                        #best_lossより同等か小さいvalid_lossが得られたらそのモデルとtokenizerをモデルとして保存する
                        if verbose:
                            print(f"epoch:{epoch} | Train Loss:{train_loss/(i+1)} | Validation loss:{valid_loss}")
                            print(f"{g_}Validation loss Decreased from {best_loss} to {valid_loss}{sr_}")

                        best_loss = valid_loss
                        torch.save(model.state_dict(),f'./model{fold}/model{fold}.bin')
                        tokenizer.save_pretrained(f'./model{fold}')
                        
        return best_loss
    
    accelerator = Accelerator()
    print(f"{accelerator.device} is used")
    
    #foldで指定されたfoldナンバーがvalid用, そのほかがtrain用になっている
    x_train,x_valid = train_data.query(f"Fold != {fold}"),train_data.query(f"Fold == {fold}")
    
    #tokenizerモデルを読み込む
    tokenizer = AutoTokenizer.from_pretrained(config['model_path'])
    #modelを読み込む
    model = Model(config['model_path'])

    #読み込んだtokenizerでx_trainを変換してdatasetとする
    train_ds = CLRPDataset(x_train,tokenizer,config['max_len'])
    #datasetをdataLoaderに読み込ませてtrain_dl(what dl stand for?)する.
    train_dl = DataLoader(train_ds,
                        batch_size = config["batch_size"],
                        shuffle=True,
                        num_workers = 4,
                        pin_memory=True,
                        drop_last=False)
    
    #validも同様に処理
    valid_ds = CLRPDataset(x_valid,tokenizer,config['max_len'])
    valid_dl = DataLoader(valid_ds,
                        batch_size = config["batch_size"],
                        shuffle=False,
                        num_workers = 4,
                        pin_memory=True,
                        drop_last=False)

    #optimizerの設定
    optimizer = optim.AdamW(model.parameters(),lr=config['lr'],weight_decay=config['wd'])
    #学習率スケジューラ設定
    lr_scheduler = get_cosine_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps= 10 * len(train_dl))

    #acceleratorでいい感じに処理できるようにする
    model,train_dl,valid_dl,optimizer,lr_scheduler = accelerator.prepare(model,train_dl,valid_dl,optimizer,lr_scheduler)

    print(f"Fold: {fold}")
    
    #best_lossでかいところから初めて
    #runの中で定義したtrain_and_evaluate_loop関数で学習をepoch数だけ繰り返す
    
    best_loss = 9999
    for epoch in range(config["epochs"]):
        print(f"Epoch Started:{epoch}")
        best_loss = train_and_evaluate_loop(train_dl,valid_dl,model,loss_fn,
                                            optimizer,epoch,fold,best_loss,
                                            valid_step=config['valid_step'],lr_scheduler=lr_scheduler)

In [ ]:
#fold数だけ実施する
for f in range(config['nfolds']):
    run(f)